<a href="https://colab.research.google.com/github/PedroNishimura/project-Spark/blob/main/project_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [670 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,691 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,401 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,680 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.lau

In [26]:
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [2]:
!tar xf /content/spark-3.5.0-bin-hadoop3.tgz

In [9]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [4]:
!pip install pyspark

In [5]:
!pip install -q findspark

In [6]:
import findspark

findspark.init()

In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("MovieAnalysis").getOrCreate()

In [12]:
from google.colab import files

arquivo = files.upload()

Saving movies.csv to movies.csv


In [ ]:
df = spark.read.csv('movies.csv', header=True, inferSchema=True)

In [17]:
df.printSchema()

root
 |-- Position: integer (nullable = true)
 |-- Const: string (nullable = true)
 |-- Created: date (nullable = true)
 |-- Modified: date (nullable = true)
 |-- Description: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- Title Type: string (nullable = true)
 |-- IMDb Rating: double (nullable = true)
 |-- Runtime (mins): integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Num Votes: integer (nullable = true)
 |-- Release Date: date (nullable = true)
 |-- Directors: string (nullable = true)



In [13]:
df.show()

+--------+---------+----------+----------+-----------+--------------------+--------------------+----------+-----------+--------------+----+--------------------+---------+------------+--------------------+
|Position|    Const|   Created|  Modified|Description|               Title|                 URL|Title Type|IMDb Rating|Runtime (mins)|Year|              Genres|Num Votes|Release Date|           Directors|
+--------+---------+----------+----------+-----------+--------------------+--------------------+----------+-----------+--------------+----+--------------------+---------+------------+--------------------+
|       1|tt4369136|2019-03-11|2019-03-11|       NULL|   Hide in the Light|https://www.imdb....|     movie|        3.7|            78|2018|    Horror, Thriller|      464|  2018-03-11|      Mikey McGregor|
|       2|tt1622566|2019-03-11|2019-03-11|       NULL|        After Effect|https://www.imdb....|     movie|        4.1|            95|2012|Action, Horror, T...|      606|  2012-02-

In [18]:
df_movie = df.select('Title', 'Year', 'Genres', 'Directors', 'Num Votes')

In [19]:
df_movie.show()

+--------------------+----+--------------------+--------------------+---------+
|               Title|Year|              Genres|           Directors|Num Votes|
+--------------------+----+--------------------+--------------------+---------+
|   Hide in the Light|2018|    Horror, Thriller|      Mikey McGregor|      464|
|        After Effect|2012|Action, Horror, T...|       David McElroy|      606|
|        The Detained|2017|Comedy, Drama, Ho...|         Blair Hayes|     1614|
|         #SquadGoals|2018|     Drama, Thriller|      Danny J. Boyle|      623|
|Await Further Ins...|2018|Horror, Mystery, ...|    Johnny Kevorkian|     9560|
|Haunting on Frate...|2018|Horror, Mystery, ...|        Brant Sersen|     1181|
|               Lasso|2017|    Horror, Thriller|          Evan Cecil|      719|
|           Extremity|2018|     Horror, Mystery|     Anthony DiBlasi|     1092|
|              Trauma|2017|Action, Drama, Ho...|      Lucio A. Rojas|     2364|
|        The Snarling|2018|Action, Comed

In [22]:
df_movie = df_movie.withColumnRenamed('Num Votes', 'Votes')

In [23]:
df_movie.filter(df_movie.Votes > 800).show()

+--------------------+----+--------------------+--------------------+-----+
|               Title|Year|              Genres|           Directors|Votes|
+--------------------+----+--------------------+--------------------+-----+
|        The Detained|2017|Comedy, Drama, Ho...|         Blair Hayes| 1614|
|Await Further Ins...|2018|Horror, Mystery, ...|    Johnny Kevorkian| 9560|
|Haunting on Frate...|2018|Horror, Mystery, ...|        Brant Sersen| 1181|
|           Extremity|2018|     Horror, Mystery|     Anthony DiBlasi| 1092|
|              Trauma|2017|Action, Drama, Ho...|      Lucio A. Rojas| 2364|
|      Mr. Malevolent|2018|      Comedy, Horror|Darin Scott, Rust...|  825|
|        The Basement|2018|              Horror|Nathan Ives, Bria...| 4137|
|Tales from the Ho...|2018|Comedy, Crime, Dr...|Darin Scott, Rust...| 2272|
|          The Toybox|2018|              Horror|           Tom Nagel| 2698|
|          Inoperable|2017|     Horror, Mystery|Christopher Lawre...|  862|
|         Ou

In [24]:
from pyspark.sql.functions import max

df_max = df_movie.agg(max('Votes').alias('max_votes')).show()

+---------+
|max_votes|
+---------+
|    91068|
+---------+



In [27]:
df_max_votes = df_movie.filter(df_movie.Votes == 91068).show()

+------+----+--------------------+--------------------+-----+
| Title|Year|              Genres|           Directors|Votes|
+------+----+--------------------+--------------------+-----+
|Jigsaw|2017|Horror, Mystery, ...|Michael Spierig, ...|91068|
+------+----+--------------------+--------------------+-----+



In [39]:
from pyspark.sql.functions import count, col, asc, desc

df_sum = df_movie.groupBy('Genres').count()

df_sum.show(truncate=False)

+---------------------------------------------------+-----+
|Genres                                             |count|
+---------------------------------------------------+-----+
|Drama, Horror, Mystery, Thriller                   |4    |
|Comedy, Fantasy, Horror, Mystery                   |1    |
|Drama, Horror, Mystery, Thriller, War              |1    |
|Action, Adventure, Drama, Sci-Fi, Thriller         |1    |
|Thriller                                           |1    |
|Drama, Horror, Mystery, Sci-Fi                     |1    |
|Action, Adventure, Comedy, Horror, Sci-Fi, Thriller|16   |
|Action, Horror, Sci-Fi, Thriller                   |5    |
|Adventure, Horror, Thriller                        |1    |
|Comedy, Horror, Thriller                           |5    |
|Mystery, Thriller                                  |1    |
|Adventure, Comedy, Horror                          |1    |
|Short, Horror                                      |2    |
|Documentary, Horror, Thriller          

In [42]:
df_sum.orderBy(col('count').desc()).show(truncate=False)

+------------------------------------------------------------+-----+
|Genres                                                      |count|
+------------------------------------------------------------+-----+
|Horror                                                      |62   |
|Horror, Thriller                                            |27   |
|Horror, Mystery, Thriller                                   |16   |
|Action, Adventure, Comedy, Horror, Sci-Fi, Thriller         |16   |
|Comedy, Horror                                              |14   |
|Drama, Horror, Thriller                                     |9    |
|Horror, Mystery                                             |9    |
|Documentary, Horror                                         |8    |
|Horror, Sci-Fi, Thriller                                    |7    |
|Action, Horror, Sci-Fi, Thriller                            |5    |
|Comedy, Horror, Thriller                                    |5    |
|Horror, Sci-Fi                   

In [45]:
df_movie.select('Title', 'Year', 'Genres', 'Directors').filter(df_movie.Year.isin(2017, 2014)).show()

df_movie.select('Title', 'Year', 'Genres', 'Directors').filter(~df_movie.Year.isin(2017, 2014)).show()

+--------------------+----+--------------------+--------------------+
|               Title|Year|              Genres|           Directors|
+--------------------+----+--------------------+--------------------+
|        The Detained|2017|Comedy, Drama, Ho...|         Blair Hayes|
|               Lasso|2017|    Horror, Thriller|          Evan Cecil|
|              Trauma|2017|Action, Drama, Ho...|      Lucio A. Rojas|
|         In Extremis|2017|    Horror, Thriller|         Steve Stone|
|          Inoperable|2017|     Horror, Mystery|Christopher Lawre...|
|              Realms|2017|     Horror, Mystery|         Daric Gates|
|5 Headed Shark At...|2017|Action, Adventure...|        Nico De Leon|
|Mega Shark vs. Me...|2014|Action, Adventure...|   Emile Edwin Smith|
|Sharknado 5: Glob...|2017|Action, Adventure...| Anthony C. Ferrante|
|         Toxic Shark|2017|Action, Adventure...|         Cole Sharpe|
|  Trailer Park Shark|2017|Action, Adventure...|         Griff Furst|
|         House Shar

In [48]:
df_movie.createOrReplaceTempView('movies')

spark.sql('''
SELECT
Year,
count(*) AS Qtd
FROM movies
GROUP BY Year
ORDER BY 1 DESC''').show(truncate=False)

+----+---+
|Year|Qtd|
+----+---+
|2023|1  |
|2022|2  |
|2020|4  |
|2019|8  |
|2018|77 |
|2017|53 |
|2016|25 |
|2015|28 |
|2014|27 |
|2013|13 |
|2012|10 |
|2011|5  |
|2010|3  |
|2008|4  |
|2007|4  |
|2006|3  |
|2005|1  |
|2004|2  |
|2002|1  |
|2001|1  |
+----+---+
only showing top 20 rows

